## Desafio DWE 2025 - Habilidades de manipulação de dados com Pyspark
- Análises propostas pelo desafio sobre o conjunto de dados contido no LakeHouse desenvolvido neste projeto.

In [0]:
import gc
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Habilidades de manipulação de dados com Pyspark") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()


# Diretórios de trabalho 
gd_dim_path = '/mnt/desafio-dwe-25/lhdw/gold/dim'
gd_fato_path = '/mnt/desafio-dwe-25/lhdw/gold/fato/VENDAS'

categoria_path = f'{gd_dim_path}/CATEGORIA'
produto_path = f'{gd_dim_path}/PRODUTO'
vendedor_path = f'{gd_dim_path}/VENDEDOR'
cidade_path = f'{gd_dim_path}/CIDADE'
pais_path = f'{gd_dim_path}/PAIS'
cliente_path = f'{gd_dim_path}/CLIENTE'


In [0]:
# Aplicação de distribuição broadcast nas dimensões que serão usadas
dim_categoria = broadcast(spark.read.format("delta").load(categoria_path))
dim_produto = broadcast(spark.read.format("delta").load(produto_path))
dim_vendedor = broadcast(spark.read.format("delta").load(vendedor_path))
dim_cidade = broadcast(spark.read.format("delta").load(cidade_path))
dim_pais = broadcast(spark.read.format("delta").load(pais_path))
dim_cliente = broadcast(spark.read.format("delta").load(cliente_path))

# Carregando Fato Vendas
fato_vendas = spark.read.format("delta").load(gd_fato_path)

# Carregando na cache o volume de dados de maior uso na análise
vendas_2018 = fato_vendas.filter("ANO = 2018")\
  .join(dim_produto, "ProdutoSK", "inner")\
  .join(dim_categoria, "CategoriaSK", "inner")

vendas_2018.cache()

Out[11]: DataFrame[CategoriaSK: bigint, ProdutoSK: bigint, VendasID: int, Quantidade: int, Desconto: double, PrecoTotal: double, DataVenda: date, DATA_ATUALIZACAO: timestamp, ANO: int, MES: int, ClienteSK: bigint, VendedorSK: bigint, ProdutoID: int, ProdutoNome: string, Preco: double, Classe: string, DataCadastro: string, Resistencia: string, EAlergico: string, ValidadeDias: double, DATA_ATUALIZACAO: timestamp, DataINI: timestamp, DataFIM: timestamp, Ativo: boolean, CategoriaID: int, NomeCategoria: string, DATA_ATUALIZACAO: timestamp, DataINI: timestamp, DataFIM: timestamp, ATIVO: boolean]

### 1 - Qual o número total de linhas carregado na tabela fato?

In [0]:
# Mostrando cabeçalho e contagem de registros Fato Vendas
fato_vendas.show(5)
print(f"Total de registros contidos no diretório Fato Vendas: {fato_vendas.count()}")

# Limpando o cache do Dataframe total no cluster e na referência
fato_vendas.unpersist()
del fato_vendas
gc.collect()

+--------+----------+--------+------------------+----------+--------------------+----+---+---------+----------+---------+
|VendasID|Quantidade|Desconto|        PrecoTotal| DataVenda|    DATA_ATUALIZACAO| ANO|MES|ClienteSK|VendedorSK|ProdutoSK|
+--------+----------+--------+------------------+----------+--------------------+----+---+---------+----------+---------+
| 1447191|        13|     0.0|          826.1812|2018-02-07|2025-02-21 18:48:...|2018|  2|    48640|        21|      155|
|  233900|         1|     0.2|49.759440000000005|2018-02-07|2025-02-21 18:48:...|2018|  2|     2782|        19|       96|
|  930189|         4|     0.0|          275.5156|2018-02-07|2025-02-21 18:48:...|2018|  2|    15286|        23|      105|
| 4792802|         7|     0.0|          132.8299|2018-02-07|2025-02-21 18:48:...|2018|  2|    26268|        10|      205|
| 6576409|        17|     0.0|492.82829999999996|2018-02-07|2025-02-21 18:48:...|2018|  2|    65099|        17|      338|
+--------+----------+---

### 2 - Qual o total de Vendas em Fev/2018 para cada Categoria?

In [0]:
# Carregando vendas de Fev/18
fev18_vendas = vendas_2018.filter("MES = 02")

# Agrupando por categoria e exibindo em formato monetário Brasileiro
print("Total de Vendas em Fev/2018 por Categoria:")
fev18_vendas.groupBy("NomeCategoria", "Ano", "Mes")\
   .agg(sum("PrecoTotal").alias("TotalVendas"))\
   .orderBy(asc("NomeCategoria"))\
   .withColumn("TotalVendas", format_number(col("TotalVendas"), 2))\
   .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), ",", "X"))\
   .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "\\.", ","))\
   .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "X", "."))\
   .select("NomeCategoria", "TotalVendas", "Ano", "Mes").show()

fev18_vendas.cache()

Total de Vendas em Fev/2018 por Categoria:
+-------------+-------------+----+---+
|NomeCategoria|  TotalVendas| Ano|Mes|
+-------------+-------------+----+---+
|    Beverages|17.559.163,88|2018|  2|
|      Cereals|20.080.823,41|2018|  2|
|  Confections|26.624.685,47|2018|  2|
|        Dairy|16.870.060,99|2018|  2|
|        Grain|15.543.903,02|2018|  2|
|         Meat|23.509.827,64|2018|  2|
|      Poultry|21.095.595,21|2018|  2|
|      Produce|17.419.229,59|2018|  2|
|      Seafood|15.741.556,97|2018|  2|
|   Shell fish|14.254.486,92|2018|  2|
|       Snails|17.908.210,32|2018|  2|
+-------------+-------------+----+---+

Out[64]: DataFrame[CategoriaSK: bigint, ProdutoSK: bigint, VendasID: int, Quantidade: int, Desconto: double, PrecoTotal: double, DataVenda: date, DATA_ATUALIZACAO: timestamp, ANO: int, MES: int, ClienteSK: bigint, VendedorSK: bigint, ProdutoID: int, ProdutoNome: string, Preco: double, Classe: string, DataCadastro: string, Resistencia: string, EAlergico: string, Validad

### 3 - Qual o total de vendas em quantidade em Mar/2018 por Vendedor?

In [0]:
# Carregando vendas de Mar/18 
mar18_vendas= vendas_2018.filter("MES = 03")

# Join vendedores e consulta agrupada de quantidades por Vendedor
print("Total de Vendas em quantidade em Mar/2018 por Vendedor:")
mar18_vendas.join(dim_vendedor, "VendedorSK", "inner")\
  .groupBy("NomeVendedor",  "Mes", "Ano")\
  .agg(count("VendasID").alias("QuantidadeVendas"))\
  .orderBy(asc("NomeVendedor"))\
  .show()

mar18_vendas.cache()

Total de Vendas em quantidade em Mar/2018 por Vendedor:
+------------------+---+----+----------------+
|      NomeVendedor|Mes| Ano|QuantidadeVendas|
+------------------+---+----+----------------+
|   Bernard L Moody|  3|2018|           14099|
|   Chadwick P Cook|  3|2018|           13993|
| Chadwick U Walton|  3|2018|           13921|
|Christine W Palmer|  3|2018|           14050|
|     Daphne X King|  3|2018|           13991|
| Darnell O Nielsen|  3|2018|           13995|
|  Desiree L Stuart|  3|2018|           14250|
|    Devon D Brewer|  3|2018|           13835|
|   Holly E Collins|  3|2018|           13871|
|   Janet K Flowers|  3|2018|           14077|
|       Jean P Vang|  3|2018|           13897|
|      Julie E Dyer|  3|2018|           14027|
|     Kari D Finley|  3|2018|           14208|
|    Katina Y Marks|  3|2018|           13837|
|    Lindsay M Chen|  3|2018|           14091|
|   Nicole T Fuller|  3|2018|           14025|
|     Pablo Y Cline|  3|2018|           13995|
|   

### 4 - Qual o total de vendas em (R$) em Fev/2018 por país?

In [0]:
# Junção de referência do país
fev18_pais = fev18_vendas\
    .join(dim_cliente, "ClienteSK", "inner")\
    .join(dim_cidade, "CidadeSK", "inner")\
    .join(dim_pais, "PaisSK", "inner")

# Agrupando por País e exibindo em formato monetário Brasileiro
fev18_pais.groupBy("PaisNome", "Mes", "Ano")\
    .agg(sum("PrecoTotal").alias("TotalVendas"))\
    .orderBy(asc("PaisNome"))\
    .withColumn("TotalVendas", format_number(col("TotalVendas"), 2))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), ",", "x"))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "\\.", ","))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "x", "."))\
    .show()

fev18_pais.unpersist()
del fev18_pais
gc.collect()

+-------------+---+----+--------------+
|     PaisNome|Mes| Ano|   TotalVendas|
+-------------+---+----+--------------+
|United States|  2|2018|206.607.543,40|
+-------------+---+----+--------------+

Out[62]: 6119

### 5 - Qual o Total de Vendas(R$) mês a mês em 2018?

In [0]:
# Agrupamento por mês de vendas e exibição em formato monetário Brasileiro
print("Total de Vendas(R$) por Mês em 2018:")
vendas_2018.groupBy("Mes", "Ano")\
    .agg(sum("PrecoTotal").alias("TotalVendas"))\
    .orderBy(asc("Mes"))\
    .withColumn("TotalVendas", format_number(col("TotalVendas"), 2))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), ",", "x"))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "\\.", ","))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "x", "."))\
    .show()



Total de Vendas(R$) por Mês em 2018:
+---+----+--------------+
|Mes| Ano|   TotalVendas|
+---+----+--------------+
|  1|2018|206.542.923,58|
|  2|2018|206.607.543,40|
|  3|2018|206.253.273,59|
+---+----+--------------+



### 6 - Qual o total de desconto mês a mês em 2018?

In [0]:
# Aplicando o valor de desconto no Dataset
vendas_2018_desconto = vendas_2018.withColumn(
    "DescontoAplicado", 
    col("Preco") * col("Quantidade") * col("Desconto")
)

# Agrupamento por mês de desconto aplicado e exibição em formato monetário Brasileiro
print("Total de desconto(R$) por Mês em 2018:")
vendas_2018_desconto.groupBy("Mes", "Ano")\
    .agg(sum("DescontoAplicado").alias("TotalDesconto"))\
    .orderBy(asc("Mes"))\
    .withColumn("TotalDesconto", format_number(col("TotalDesconto"), 2))\
    .withColumn("TotalDesconto", regexp_replace(col("TotalDesconto"), ",", "x"))\
    .withColumn("TotalDesconto", regexp_replace(col("TotalDesconto"), "\\.", ","))\
    .withColumn("TotalDesconto", regexp_replace(col("TotalDesconto"), "x", "."))\
    .show()

vendas_2018_desconto.unpersist()
del vendas_2018_desconto
gc.collect()

Total de desconto(R$) por Mês em 2018:
+---+----+-------------+
|Mes| Ano|TotalDesconto|
+---+----+-------------+
|  1|2018| 6.315.273,39|
|  2|2018| 6.389.943,53|
|  3|2018| 6.324.096,74|
+---+----+-------------+

Out[69]: 21227

### 7 - Qual a variação %(MoM) de Total de vendas (R$) de Mar/2018 para Fev/2018?

In [0]:
# Coletando total de vendas em Fevereiro e Março de 2018
vendas_fev = fev18_vendas.agg(sum("PrecoTotal").alias("VendasFEV"))
vendas_mar = mar18_vendas.agg(sum("PrecoTotal").alias("VendasMAR"))

# Calculando a variação em % nas vendas em Fev e Mar
vendas_var = vendas_fev.withColumn("VendasMAR", lit(vendas_mar.collect()[0][0]))\
    .withColumn("Variacao(%)", (((col("VendasMAR")) / col("VendasFEV")) - 1) * 100)\
    .withColumn("Variacao(%)", format_string("%.2f", col("Variacao(%)")))

# Exibindo os valores monetários de Fev em formato BR
vendas_var = vendas_var.withColumn("VendasFEV", format_number(col("VendasFEV"), 2))\
    .withColumn("VendasFEV", regexp_replace(col("VendasFEV"), ",", "x"))\
    .withColumn("VendasFEV", regexp_replace(col("VendasFEV"), "\\.", ","))\
    .withColumn("VendasFEV", regexp_replace(col("VendasFEV"), "x", "."))

# Exibindo os valores monetários de Mar em formato BR
vendas_var = vendas_var.withColumn("VendasMAR", format_number(col("VendasMAR"), 2))\
    .withColumn("VendasMAR", regexp_replace(col("VendasMAR"), ",", "x"))\
    .withColumn("VendasMAR", regexp_replace(col("VendasMAR"), "\\.", ","))\
    .withColumn("VendasMAR", regexp_replace(col("VendasMAR"), "x", "."))

# Exibindo resultado (Trocando ordem das colunas para fazer sentido)
print("Variação %(MoM) de Total de vendas (R$) de Mar/2018 para Fev/2018:")
vendas_var.select("VendasMAR", "VendasFEV", "Variacao(%)").show()

vendas_fev.unpersist()
vendas_mar.unpersist()
vendas_var.unpersist()
del vendas_fev, vendas_mar,  vendas_var
gc.collect()

Variação %(MoM) de Total de vendas (R$) de Mar/2018 para Fev/2018:
+--------------+--------------+-----------+
|     VendasMAR|     VendasFEV|Variacao(%)|
+--------------+--------------+-----------+
|206.253.273,59|206.607.543,40|      -0.17|
+--------------+--------------+-----------+

Out[96]: 13177

### 8 - Qual o Top 10 produtos com maior Valor de vendas (R$) em Fev/2018?

In [0]:
# Exibindo top 10 com valores monetarios em formato BR
print("Top 10 produtos com maior Valor de vendas (R$) em Fev/2018:")
fev18_vendas.groupBy("ProdutoNome")\
    .agg(sum("PrecoTotal").alias("TotalVendas"))\
    .orderBy(desc("TotalVendas"))\
    .withColumn("TotalVendas", format_number(col("TotalVendas"), 2))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), ",", "x"))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "\\.", ","))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "x", "."))\
    .show(10)

+--------------------+-----------+
|         ProdutoNome|TotalVendas|
+--------------------+-----------+
|           Tia Maria| 993.120,99|
|Puree - Passion F...| 961.500,84|
|  Bread - Multigrain| 931.051,22|
| Beer - Rickards Red| 900.530,87|
|Pork - Hock And F...| 895.106,43|
|Soup Knorr Chili ...| 883.830,80|
|Bread - Calabrese...| 881.217,84|
|       Vanilla Beans| 880.190,34|
| Beef - Inside Round| 872.808,08|
| Tuna - Salad Premix| 868.599,36|
+--------------------+-----------+
only showing top 10 rows



### 9 - Retorne o % (Share*) de Total de vendas (R$), por Categoria em Fev/2018?

In [0]:
# Consultando o valor total de vendas de Fev/18
total_vendas = fev18_vendas.agg(sum("PrecoTotal")).collect()[0][0]

# Agrupando o Share por Categoria com base no total de vendas
share_vendas18 = fev18_vendas.groupBy("NomeCategoria")\
    .agg(sum("PrecoTotal").alias("TotalVendas"))\
    .withColumn("Share(%)", (((col("TotalVendas")) / total_vendas)) * 100)\
    .orderBy(desc("Share(%)"))\
    .withColumn("Share(%)", format_string("%.2f", col("Share(%)")))

# Exibindo o resultado com o formato monetario BR
print("% (Share*) de Total de vendas (R$), por Categoria em Fev/2018:")
share_vendas18.withColumn("TotalVendas", format_number(col("TotalVendas"), 2))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), ",", "x"))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "\\.", ","))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "x", "."))\
    .show()

share_vendas18.unpersist()
del share_vendas18
gc.collect()

% (Share*) de Total de vendas (R$), por Categoria em Fev/2018:
+-------------+-------------+--------+
|NomeCategoria|  TotalVendas|Share(%)|
+-------------+-------------+--------+
|  Confections|26.624.685,47|   12.89|
|         Meat|23.509.827,64|   11.38|
|      Poultry|21.095.595,21|   10.21|
|      Cereals|20.080.823,41|    9.72|
|       Snails|17.908.210,32|    8.67|
|    Beverages|17.559.163,88|    8.50|
|      Produce|17.419.229,59|    8.43|
|        Dairy|16.870.060,99|    8.17|
|      Seafood|15.741.556,97|    7.62|
|        Grain|15.543.903,02|    7.52|
|   Shell fish|14.254.486,92|    6.90|
+-------------+-------------+--------+

Out[124]: 13320

### 10 - Retorne o Ticket médio de Total de Vendas por categoria no ano de Jan/2018?

In [0]:
# Trazendo a quantidade de vendas por categoria de Jan/18
jan18_vendas_quantidade = vendas_2018.filter("MES = 01")\
    .groupBy("NomeCategoria").agg(count("VendasID").alias("Quantidade"))

# Unindo a quantidade de vendas ao total de vendas para calcular o Ticket Medio
ticket_jan18 =  vendas_2018.filter("MES = 01")\
    .groupBy("NomeCategoria").agg(sum("PrecoTotal").alias("TotalVendas"))\
    .join(jan18_vendas_quantidade, "NomeCategoria", "inner")\
    .withColumn("TicketMedio", (col("TotalVendas") / col("Quantidade")))

# Exibição do resultado no formato monetario BR
print("Ticket médio de Total de Vendas por categoria no ano de Jan/2018:")
ticket_jan18.orderBy(asc("NomeCategoria"))\
    .withColumn("TotalVendas", format_number(col("TotalVendas"), 2))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), ",", "x"))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "\\.", ","))\
    .withColumn("TotalVendas", regexp_replace(col("TotalVendas"), "x", "."))\
    .withColumn("TicketMedio", format_number(col("TicketMedio"), 2))\
    .withColumn("TicketMedio", regexp_replace(col("TicketMedio"), ",", "x"))\
    .withColumn("TicketMedio", regexp_replace(col("TicketMedio"), "\\.", ","))\
    .withColumn("TicketMedio", regexp_replace(col("TicketMedio"), "x", "."))\
    .select("NomeCategoria", "TotalVendas", "TicketMedio").show()

jan18_vendas_quantidade.unpersist()
ticket_jan18.unpersist()
del jan18_vendas_quantidade, ticket_jan18
gc.collect()


Ticket médio de Total de Vendas por categoria no ano de Jan/2018:
+-------------+-------------+-----------+
|NomeCategoria|  TotalVendas|TicketMedio|
+-------------+-------------+-----------+
|    Beverages|17.521.891,00|     645,90|
|      Cereals|20.375.501,84|     635,88|
|  Confections|26.694.946,97|     657,67|
|        Dairy|16.626.946,66|     673,67|
|        Grain|15.457.952,32|     774,56|
|         Meat|23.684.096,25|     664,16|
|      Poultry|20.761.846,14|     622,15|
|      Produce|17.467.967,81|     569,82|
|      Seafood|15.974.324,46|     617,60|
|   Shell fish|14.344.095,02|     557,29|
|       Snails|17.633.355,13|     674,94|
+-------------+-------------+-----------+

Out[127]: 50394

### Fim Análise - Eliminando cache e memória 

In [0]:
dim_categoria.unpersist()
dim_produto.unpersist()
dim_vendedor.unpersist()
dim_cidade.unpersist()
dim_pais.unpersist()
dim_cliente.unpersist()

vendas_2018.unpersist()
fev18_vendas.unpersist()
mar18_vendas.unpersist()

del dim_categoria, dim_produto, dim_vendedor, dim_cidade, dim_pais, dim_cliente
del vendas_2018, fev18_vendas, mar18_vendas

gc.collect()

Out[128]: 39479